In [57]:
def read_file(filepath):
    with open(filepath) as f:
        str_text = f.read()
    
    return str_text

In [58]:
# read_file('moby_dick_four_chapters.txt')

In [59]:
import spacy

In [60]:
nlp = spacy.load('en', disable=['parser', 'tagger', 'ner'])

In [61]:
nlp.max_length = 1198623

In [62]:
def separate_punc(doc_text):
    return [token.text.lower() for token in nlp(doc_text) if token.text not in '\n\n \n\n\n!"-#$%&()--.*+,-/:;<=>?@[\\]^_`{|}~\t\n ']

In [63]:
d = read_file('moby_dick_four_chapters.txt')

In [64]:
tokens = separate_punc(d)

In [65]:
len(tokens) # words here

11338

In [66]:
#25 words --> network predicts word #26

In [67]:
train_len = 25 + 1

text_sequences = []

for i in range(train_len, len(tokens)):
    seq = tokens[i-train_len:i]

    text_sequences.append(seq)

In [68]:
type(text_sequences)

list

In [69]:
# text_sequences[0]
# text_sequences[1] # sequences, each sequence is one token over (+1 beginign and end)

' '.join(text_sequences[0])

'call me ishmael some years ago never mind how long precisely having little or no money in my purse and nothing particular to interest me on'

In [70]:
' '.join(text_sequences[1])

'me ishmael some years ago never mind how long precisely having little or no money in my purse and nothing particular to interest me on shore'

In [71]:
' '.join(text_sequences[2])

'ishmael some years ago never mind how long precisely having little or no money in my purse and nothing particular to interest me on shore i'

In [72]:
# format text sequence into a numerical sequence for keras
from keras.preprocessing.text import Tokenizer

In [73]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_sequences)

In [74]:
sequences = tokenizer.texts_to_sequences(text_sequences)

In [75]:
# now we have our converted numeric sequences
# sequences[0] 

In [76]:
# each number is actually a word (token)
# tokenizer.index_word

In [77]:
for i in sequences[0]:
    print(f"{i} : {tokenizer.index_word[i]}")

956 : call
14 : me
263 : ishmael
51 : some
261 : years
408 : ago
87 : never
219 : mind
129 : how
111 : long
954 : precisely
260 : having
50 : little
43 : or
38 : no
315 : money
7 : in
23 : my
546 : purse
3 : and
150 : nothing
259 : particular
6 : to
2712 : interest
14 : me
24 : on


In [78]:
# see how many times words appears in our sequence
# tokenizer.word_counts

In [79]:
vocabulary_size = len(tokenizer.word_counts) # number of unique words in all this 4 chapters document that we got

In [80]:
type(sequences)

list

In [81]:
# turn the sequence into a numpy matrix format, to create a train split with it
import numpy as np

In [82]:
sequences = np.array(sequences)

In [83]:
sequences

array([[ 956,   14,  263, ..., 2712,   14,   24],
       [  14,  263,   51, ...,   14,   24,  957],
       [ 263,   51,  261, ...,   24,  957,    5],
       ...,
       [ 952,   12,  166, ...,  262,   53,    2],
       [  12,  166, 2711, ...,   53,    2, 2717],
       [ 166, 2711,    3, ...,    2, 2717,   26]])

In [84]:
# part 2: Create our data (X, y)
from keras.utils import to_categorical

In [85]:
sequences[:, :-1] # each row of words (features)

array([[ 956,   14,  263, ...,    6, 2712,   14],
       [  14,  263,   51, ..., 2712,   14,   24],
       [ 263,   51,  261, ...,   14,   24,  957],
       ...,
       [ 952,   12,  166, ...,   11,  262,   53],
       [  12,  166, 2711, ...,  262,   53,    2],
       [ 166, 2711,    3, ...,   53,    2, 2717]])

In [86]:
sequences[:,-1] # last word of each row (label)

array([  24,  957,    5, ...,    2, 2717,   26])

In [87]:
X = sequences[:, :-1] # features

In [88]:
y = sequences[:,-1] # label

In [89]:
y = to_categorical(y, num_classes=vocabulary_size+1)

In [90]:
X.shape # (sequences, words per sequence)

(11312, 25)

In [91]:
seq_len = X.shape[1]

In [92]:
# Create our model with the data
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding # LSTM: For the sequences, Embedding: for the words

In [93]:
# function to create models
def create_model(vocabulary_size, seq_len):

    model = Sequential()

    model.add(Embedding(vocabulary_size, seq_len, input_length=seq_len))
    # LSTM layer chooses amount of neurons, recommended to have as amount a multiple of seq_len
    model.add(LSTM(seq_len*2, return_sequences=True)) # first param should be multiple of seq_len (Recomended)
    model.add(LSTM(seq_len*2))

    model.add(Dense(50, activation='relu'))
    model.add(Dense(vocabulary_size, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    model.summary()

    return model

In [94]:
model = create_model(vocabulary_size+1, seq_len) # creating our model

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 25, 25)            67950     
_________________________________________________________________
lstm_2 (LSTM)                (None, 25, 50)            15200     
_________________________________________________________________
lstm_3 (LSTM)                (None, 50)                20200     
_________________________________________________________________
dense_2 (Dense)              (None, 50)                2550      
_________________________________________________________________
dense_3 (Dense)              (None, 2718)              138618    
Total params: 244,518
Trainable params: 244,518
Non-trainable params: 0
_________________________________________________________________


In [95]:
# training our model

In [96]:
# to save files for later and load them
from pickle import dump, load

In [97]:
model.fit(
    X, y,
    batch_size=128, # how many sequences do you want to pass at a time
    epochs=2, # number of tranings in the execution
    verbose=1 # output report
)

Epoch 1/2
89/89 [==============================] - 6s 39ms/step - loss: 7.4443 - accuracy: 0.0360
Epoch 2/2
89/89 [==============================] - 4s 41ms/step - loss: 6.3543 - accuracy: 0.0521


In [98]:
# saving our model
model.save('my_mobydick_model.h5')

In [99]:
# tokenizer from Keras with all the vocabsulary of the documents
dump(tokenizer,open('my_simpletokenizer','wb')) # saving our token

In [100]:
# part 3: Generate the next sequence of words by predictions
from keras.preprocessing.sequence import pad_sequences

In [109]:
def generate_text(model, tokenizer, seq_len, seed_text, num_gen_words):

    output_text = []

    input_text = seed_text

    for i in range(num_gen_words):

        # take input text and encode it to be a sequence
        encoded_text = tokenizer.texts_to_sequences([input_text])[0] 

        # pad the text: in case that is too long or too short, we adapt it   
        pad_encoded = pad_sequences([encoded_text], maxlen=seq_len, truncating='pre')

        #  predict the class probabilities for each word
        # this method will throw the entire vocabulary and asign a probability to the most likely next word
        pred_word_ind = model.predict_classes(pad_encoded, verbose=0)[0]
  
        # get the actual matched word
        pred_word = tokenizer.index_word[pred_word_ind]

        # add the predicted word in the original input text
        input_text += ' '+pred_word

        # adding the predicted word into the list
        output_text.append(pred_word)
    
    # return all the predicted words
    return ' '.join(output_text)

In [102]:
# text_sequences[0]

In [103]:
import random
random.seed(101)
random_pick = random.randint(0, len(text_sequences))

In [104]:
random_seed_text = text_sequences[random_pick]

In [105]:
random_seed_text

['thought',
 'i',
 'to',
 'myself',
 'the',
 'man',
 "'s",
 'a',
 'human',
 'being',
 'just',
 'as',
 'i',
 'am',
 'he',
 'has',
 'just',
 'as',
 'much',
 'reason',
 'to',
 'fear',
 'me',
 'as',
 'i',
 'have']

In [106]:
seed_text = ' '.join(random_seed_text)

In [107]:
seed_text

"thought i to myself the man 's a human being just as i am he has just as much reason to fear me as i have"

In [110]:
generate_text(model, tokenizer, seq_len, seed_text, num_gen_words=25)

'the the the the the the the the the the the the the the the the the the the the the the the the the'

In [111]:
# load example model with 60% accuracy
from keras.models import load_model

In [112]:
model = load_model('epochBIG.h5')

In [114]:
tokenizer = load(open('epochBIG', 'rb'))

In [115]:
generate_text(model, tokenizer, seq_len, seed_text, num_gen_words=25)

"to be seen there was no bad olfactories my own letter was cheerily listening over his hearers who 's more can go have a wearing"